In [17]:
import requests
from PIL import Image
import io
import tensorflow as tf
import torch
import datetime
from torch.utils.tensorboard import SummaryWriter
from pathlib import Path
from datasets import DatasetDict
import time




In [3]:
from datasets import load_dataset

dataset = load_dataset("stochastic/random_streetview_images_pano_v0.0.2")

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 11054/11054 [00:26<00:00, 413.90 examples/s]


In [18]:
dataset

train_testvalid = dataset['train'].train_test_split(test_size=0.4)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

datasets = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']
})

# TODO: need to do the train/test/validate split here

# want image_data_train, image_data_test, image_data_validate

# len(dataset['train'])
# dataset['train'][2]['image']
# filtered_data = dataset.filter(lambda example: "x" in example["text"])


In [5]:
country_codes = ["ZA","KR","AR","BW","GR","SK","HK","NL","PE","AU","KH","LT","NZ","RO","MY","SG","AE","FR","ES","IT","IE","LV","IL","JP","CH","AD","CA","RU","NO","SE","PL","TW","CO","BD","HU","CL","IS","BG","GB","US","SI","BT","FI","BE","EE","SZ","UA","CZ","BR","DK","ID","MX","DE","HR","PT","TH"]
country_dict = {}
for i in range(len(country_codes)):
    country_dict[country_codes[i]] = i
    
def country_to_label(country):
    label = [0]*len(country_codes)
    label[country_dict[country]] = 1
    return label

In [11]:
# referenced: https://blog.paperspace.com/convolutional-autoencoder/
# autoencoder classes (CREDIT: LARGELY TAKEN FROM 6_AUTOENCODER NOTEBOOK, but encoder and decoder architectures modified to be convolutional)
# should only have Encoder that has a latent dimension of 50 - corresponding to country weights

class MLPEncoder(torch.nn.Module):

    def __init__(self,
                 number_of_hidden_layers: int,
                 latent_size: int,
                 hidden_size: int,
                 input_size: int,
                 activation: torch.nn.Module):
        """Construct a simple MLP decoder"""

        super().__init__()

        self.latent_size = latent_size
        assert number_of_hidden_layers >= 0, "Decoder number_of_hidden_layers must be at least 0"

        layers = []
        in_channels = 3
        out_channels = 16
        layers.append(torch.nn.Conv2d(in_channels, #in_channels
                                      out_channels, #out_channels
                                      3, #kernel_size
                                      stride=2, #stride 
                                      padding=1
                                      ))
        layers.append(activation)
        # 32 x 32
        layers.append(torch.nn.Conv2d(out_channels, #in_channels
                                      out_channels, #out_channels
                                      3, #kernel_size
                                      stride=1, #stride 
                                      padding=1
                                      ))
        layers.append(activation)
        layers.append(torch.nn.Conv2d(out_channels, #in_channels
                                      out_channels * 2, #out_channels
                                      3, #kernel_size
                                      stride=2,  #stride
                                      padding=1 
                                      ))
        layers.append(activation)
        #16 x 16
        layers.append(torch.nn.Conv2d(out_channels * 2, #in_channels
                                      out_channels * 2, #out_channels
                                      3, #kernel_size
                                      stride=1,  #stride 
                                      padding=1
                                      ))
        layers.append(activation)
        # 8x8 feature maps
        # 64 out channels
        layers.append(torch.nn.Conv2d(out_channels * 2, #in_channels
                                      out_channels*4, #out_channels
                                      3, #kernel_size
                                      stride=2,  #stride 
                                      padding=1
                                      ))
        layers.append(activation)
        # input is now 8x8 feature maps for out_channels*4 channels.
        layers.append(torch.nn.Flatten())
        #flatten to latent_size 
        layers.append(torch.nn.Linear(4*out_channels*8*8 , # features in
                                      latent_size # features out
                                      ))
        layers.append(activation)
        self.net = torch.nn.Sequential(*layers)

    def forward(self, x: torch.Tensor):
        #return torch.rand(1,self.latent_size)
        return self.net(x)


In [16]:
# define our training parameters and model
hidden_layers = 4
hidden_size = 30


latent_size = 50

## this might need to change
input_size = 64
lr = 0.001
# lambda weight for classifier's loss
lamb = 1


# fix random seed
torch.manual_seed(0)

# select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLPEncoder( number_of_hidden_layers=hidden_layers,
                 latent_size=latent_size,
                 hidden_size=hidden_size,
                 input_size=input_size,
                 activation=torch.nn.ReLU()).to(device)

# use an optimizer to handle parameter updates
opt = torch.optim.Adam(model.parameters(), lr=lr)

# save all log data to a local directory
run_dir = "logs"

# to clear out TensorBoard and start totally fresh, we'll need to
# remove old logs by deleting them from the directory
!rm -rf ./logs/

# timestamp the logs for each run so we can sort through them
run_time = datetime.datetime.now().strftime("%I:%M%p on %B %d, %Y")

# initialize a SummaryWriter object to handle all logging actions
logger = SummaryWriter(log_dir=Path(run_dir) / run_time, flush_secs=20)

In [ ]:
# (***credit***: mostly taken from provided notebook )
# training
calc_valid = False
epochs = 100
# batch_size = 0.2
# each batch is now going to go over one complete image with its adversarial examples
train_batch_size = 5/len(image_data_train)
valid_batch_size = 5/len(image_data_valid)
test_batch_size = len()
report_every = 1
start_time = time.time()
loss_history = []
valid_history = []
acc_history = []
valid_acc_history = []

# split this into batches
batched_image_data_train = torch.split(image_data_train, int(train_batch_size * len(image_data_train)))
batched_image_data_valid = torch.split(image_data_valid, int(valid_batch_size * len(image_data_valid)))
batched_image_data_test = torch.split(image_data_test, int(test_batch_size * len(image_data_test)))
for epoch in range(epochs):

    # weight batch losses/scores proportional to batch size
    iter_count = 0
    valid_iter_count = 0
    loss_epoch = 0
    class_accuracy_epoch = 0
    valid_loss_epoch = 0
    valid_accuracy_epoch = 0
    ###
    ### IMAGE_DATA_TRAIN is training data, shape is 610 x 3 x 64 x 64
    ### 
    
   # print(batched_image_data_train[0][0].shape)

    # test with literally only one image
    #batched_image_data_train = [batched_image_data_train[0][0].unsqueeze(0)]
    for batch_idx, batch_data in enumerate(batched_image_data_train):

        # update the model that classifies the emoji
        # how to know which emoji we are currently trying to classify
        # batch index?
        # classification is made below
        # need to implement backprop for the model to update?

        # we only care about inputs, not labels
        x_real = batch_data.float()
       
        # print(x_real)
        # flatten input images and move to device\
        # *****
        x_real = x_real / 255
        # plot x_real later to see if this is correct
        x_real = x_real.to(device)
        model.zero_grad()

        # train on a batch of inputs
        x_reconstructed, classification = model(x_real)

       

        true_label_tensor = torch.tensor(labels['train'][batch_idx]).repeat(5,1).float()
        loss_batch = loss([x_real,classification], [x_reconstructed, true_label_tensor])
        loss_batch.backward()
        opt.step()

        # log loss
        loss_epoch += loss_batch.detach().item() * train_batch_size
        iter_count += train_batch_size

        # apply the .5 threshold to the classification
        true_class = int(true_label_tensor[0])
        c = 1 if (classification.mean() > 0.5) else 0   
    

        # classification accuracy
        acc = (true_class == c) * train_batch_size
        class_accuracy_epoch += acc

        # print(f"true: {true_class}")
        # print(f"pred: {classification.mean()}")
        # print(f"acc: {acc}")

    # plot loss
    loss_epoch /= iter_count
    class_accuracy_epoch /= iter_count
   # print(iter_count)
    logger.add_scalar("mse_loss", loss_epoch, epoch)
    loss_history.append(loss_epoch)
    acc_history.append(class_accuracy_epoch)
 
            
            # logger.add_scalar("mse_loss_valid", valid_loss_epoch, epoch)
    # # plot example generated images
    # with torch.no_grad():
    #     reconstructed_batch = model(example_batch.reshape(batch_size, -1)).reshape(batch_size, 1, image_size, image_size)
    #     logger.add_image("reconstructed_images", make_grid(reconstructed_batch, math.floor(math.sqrt(batch_size)), title="Reconstructed Images"), epoch)
        # calculate validation loss

    with torch.no_grad():
        for batch_idx, batch_data in enumerate(batched_image_data_valid):
            x_real = batch_data.float()
            x_real = x_real / 255
            x_real = x_real.to(device)
            x_reconstructed, classification = model(x_real)

            true_label_tensor = torch.tensor(labels['valid'][batch_idx]).repeat(5,1).float()
            valid_loss = loss([x_real,classification], [x_reconstructed, true_label_tensor])
            valid_loss_epoch += valid_loss.detach().item() * valid_batch_size

            
                # apply the .5 threshold to the classification
            true_class = int(true_label_tensor[0])
            c = 1 if (classification.mean() > 0.5) else 0   

            

            # classification accuracy
            valid_acc = (true_class == c) * valid_batch_size
            valid_accuracy_epoch += valid_acc
            valid_iter_count += valid_batch_size
            # print(f"true: {true_class}")
            # print(f"pred: {classification.mean()}")
            # print(f"acc: {valid_acc}")
        valid_loss_epoch /= valid_iter_count
        valid_history.append(valid_loss_epoch)
        valid_accuracy_epoch /= valid_iter_count
        valid_acc_history.append(valid_accuracy_epoch)

    if (epoch + 1) % report_every == 0:
        mins = (time.time() - start_time) / 60
        print(f"Epoch: {epoch + 1:5d}\tMSE Loss: {loss_epoch :6.4f}\t in {mins:5.1f}min")
        print()